In [11]:
from PIL import Image
import qrcode
import numpy as np

def encode_lsb(cover_image, qr_image, strength=3):
    # Convert images to numpy arrays
    cover_data = np.asarray(cover_image)
    qr_data = np.asarray(qr_image.convert('1'))  # Convert QR code to binary

    # Flatten the arrays
    cover_flat = cover_data.flatten()
    qr_flat = qr_data.flatten()

    # Check if the QR code size exceeds the cover image size
    if len(qr_flat) > len(cover_flat):
        raise ValueError("QR code size exceeds the cover image size")

    # Encode the QR code data into the least significant bits of the cover image
    encoded_flat = cover_flat.copy()
    for i, bit in enumerate(qr_flat):
        mask = (2 ** strength) - 1
        if bit:
            encoded_flat[i] = (encoded_flat[i] & ~mask) | ((2 ** (strength - 1)) - 1)
        else:
            encoded_flat[i] = encoded_flat[i] & ~mask

    # Reshape the encoded data to the original image shape
    encoded_data = encoded_flat.reshape(cover_data.shape)

    # Convert the encoded data back to an image
    encoded_image = Image.fromarray(encoded_data.astype('uint8'))

    return encoded_image

# Generate the QR code data
qr = qrcode.QRCode(
    version=2,  # Reduce the version
    error_correction=qrcode.constants.ERROR_CORRECT_M,  # Use a lower error correction level
    box_size=20,
    border=4,
)
qr_link = 'https://freelancer-auto-bidding.netlify.app/'
qr.add_data(qr_link)
qr.make(fit=True)
qr_image = qr.make_image(fill_color="black", back_color="white")

# Load the cover image
cover_image = Image.open("/home/waseem/project/projects/art_scanner/IMG-20200303-WA0007.jpg")

# Encode the QR code into the cover image using LSB
encoded_image = encode_lsb(cover_image, qr_image, strength=4)  # Increase the embedding strength

# Resize the encoded image for mobile display
encoded_image = encoded_image.resize((800, 600), resample=Image.LANCZOS)

# Create a new transparent image with the same size as the encoded image
transparent_image = Image.new("RGBA", encoded_image.size, color=(0, 0, 0, 0))

# Calculate the position to paste the QR code
qr_width, qr_height = qr_image.size
left = (transparent_image.width - qr_width) // 2
top = (transparent_image.height - qr_height) // 2

# Paste the QR code onto the transparent image
transparent_image.paste(qr_image, (left, top), mask=qr_image)

# Combine the encoded image and the transparent image with the QR code
final_image = Image.alpha_composite(encoded_image.convert("RGBA"), transparent_image)

# Save the final image with the embedded QR code
final_image.save("final_qr_image.png", quality=90, optimize=True)

# Print the original QR code link
print(f"The QR code link is: {qr_link}")

The QR code link is: https://freelancer-auto-bidding.netlify.app/


In [12]:
from PIL import Image
from pyzbar.pyzbar import decode

# Load the image containing the encoded QR code
uploaded_image_path = "/home/waseem/project/projects/art_scanner/exploration_file/final_qr_image.png"  # Replace with the path to your uploaded image
uploaded_image = Image.open(uploaded_image_path)

# Decode the QR code from the uploaded image
decoded_qr_codes = decode(uploaded_image)

# If QR code(s) are found in the image, print the decoded data
if decoded_qr_codes:
    for qr_code in decoded_qr_codes:
        qr_data = qr_code.data.decode("utf-8")
        print("Found QR code with data:", qr_data)
else:
    print("No QR code found in the uploaded image.")


No QR code found in the uploaded image.
